# Test API Calls

In [53]:
import os
from dotenv import load_dotenv
import requests
load_dotenv()

True

### Auth

In [64]:
# Import the auth helper
import auth_helper as auth

# Initialize authentication (uses saved session or prompts for OTP)
if auth.init():
    print("🎉 Ready to make authenticated requests!")
else:
    print("❌ Authentication failed")

⚠️ Session expired
📧 Sending OTP to test@tristansinclair.dev...
✅ OTP sent! Check your email
🔐 Verifying OTP...
✅ Sign in successful!
✅ Session saved
🎉 Ready to make authenticated requests!


In [63]:
# refresh the user token
auth.refresh_token()

AttributeError: module 'auth_helper' has no attribute 'refresh_token'

In [65]:
# Make authenticated requests easily
response = auth.request("/get_user")
if response and response.ok:
    print("User info:", response.json())

User info: {'authenticated': True, 'user_id': '371c2f1c-f469-447e-9ab8-7921302fb49c', 'email': 'test@tristansinclair.dev', 'created_at': '2025-08-07T01:35:16.656512+00:00', 'metadata': {'email': 'test@tristansinclair.dev', 'email_verified': True, 'phone_verified': False, 'sub': '371c2f1c-f469-447e-9ab8-7921302fb49c'}}


### Making other API calls

In [66]:
# Example: GET request
response = auth.request("/health")
if response and response.ok:
    print("Health check:", response.json())


Health check: {'status': 'healthy', 'services': {'database': {'status': 'up'}, 'checkpointer': {'status': 'up'}}}


### Create a new thread

In [37]:
# Create a new thread
response = auth.request("/generate-thread-id", method="POST")

if response and response.status_code == 201:
    thread_data = response.json()
    thread_id = thread_data["thread_id"]
    print(f"✅ Created new thread: {thread_id}")
else:
    print(f"❌ Failed to create thread: {response.status_code if response else 'No response'}")
    if response:
        print(f"   Error: {response.text}")

✅ Created new thread: 6c6ef5a1-20d8-496a-a4cd-e68e37e5aefc


### Send a chat message to a thread

In [76]:
# Send a message to the specific thread
thread_id = "6c6ef5a1-20d8-496a-a4cd-e68e37e5aefc"
message = "Hello! Who are you?"

response = auth.request(
    f"/threads/{thread_id}/chat",
    method="POST",
    data={"message": message}
)

if response and response.ok:
    print(f"✅ Message sent successfully to thread {thread_id}")
    print("Response content type:", response.headers.get("content-type"))
    
    # Since this is a streaming endpoint, let's read the stream
    print("\n📨 Streaming response:")
    for line in response.iter_lines(decode_unicode=True):
        if line.strip():
            print(line)
else:
    print(f"❌ Failed to send message: {response.status_code if response else 'No response'}")
    if response:
        print(f"   Error: {response.text}")

✅ Message sent successfully to thread 6c6ef5a1-20d8-496a-a4cd-e68e37e5aefc
Response content type: text/event-stream

📨 Streaming response:
data: {"type": "state_update", "state": {"messages": [{"id": "240797d9-544f-4643-9a34-863d8aa722c4", "type": "human", "content": "Hello! How are you today?", "artifact": null}, {"id": "7471e56e-c4bd-4b68-9a9b-74ed5490a9d6", "type": "human", "content": "Hello! How are you today?", "artifact": null}, {"id": "385d3352-05a3-4b20-a627-28bb7697aff8", "type": "human", "content": "Hello! How are you today?", "artifact": null}, {"id": "c3535096-ebf1-4498-9e4a-ac693149751f", "type": "ai_message", "content": "I'm here to help with student loan management and Candidly's available services. How can I assist you with your student loans today?", "artifact": null}, {"id": "346e99b6-4749-4286-8d06-c2fc75e819da", "type": "human", "content": "Hello! How are you today?", "artifact": null}, {"id": "4fa95d74-d7e8-4f55-9cc6-f7552a7f3e5c", "type": "human", "content": "Hell

### Initialize LangGraph PostgreSQL Checkpointing (One-time Setup)

In [ ]:
import asyncio
import os
from dotenv import load_dotenv
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver

# Load environment variables
load_dotenv()

async def setup_checkpointer():
    """
    One-time setup to initialize LangGraph checkpointer tables in PostgreSQL.
    This creates the necessary tables for storing conversation state and checkpoints.
    """
    # Build database URL from individual environment variables
    user = os.getenv("POSTGRES_USERNAME")
    password = os.getenv("POSTGRES_PASSWORD")
    host = os.getenv("POSTGRES_HOST")
    port = os.getenv("POSTGRES_PORT", "5432")
    database = os.getenv("POSTGRES_DB")
    
    # Construct the database URL (psycopg format for AsyncPostgresSaver)
    psycopg_connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    
    print(f"📡 Connecting to: {host}:{port}/{database}")
    
    try:
        # Create AsyncPostgresSaver instance using from_conn_string
        async with AsyncPostgresSaver.from_conn_string(psycopg_connection_string) as checkpointer:
            # Setup the checkpointer tables
            await checkpointer.setup()
            print("✅ LangGraph checkpointer tables initialized successfully!")
            print("   Tables created:")
            print("   - checkpoints")
            print("   - checkpoint_blobs") 
            print("   - checkpoint_writes")
            print("   - checkpoint_metadata")
            
    except Exception as e:
        print(f"❌ Failed to setup checkpointer: {e}")
        import traceback
        traceback.print_exc()

# Run the setup
await setup_checkpointer()
print("\n⚠️ This is a one-time setup. You don't need to run this cell again unless you're setting up a new database.")

### Alternative: Check if checkpointer tables exist

In [59]:
import asyncio
from sqlalchemy.ext.asyncio import create_async_engine
from sqlalchemy import text

async def check_checkpointer_tables():
    """
    Check if LangGraph checkpointer tables exist in the database.
    """
    # Build database URL from individual environment variables
    user = os.getenv("POSTGRES_USERNAME")
    password = os.getenv("POSTGRES_PASSWORD")
    host = os.getenv("POSTGRES_HOST")
    port = os.getenv("POSTGRES_PORT")
    database = os.getenv("POSTGRES_DB")
    
    # Construct the database URL (matching backend format)
    alchemy_connection_string = f"postgresql+asyncpg://{user}:{password}@{host}:{port}/{database}"
    
    print(f"📡 Checking tables in: {host}:{port}/{database}")
    
    engine = create_async_engine(alchemy_connection_string)
    
    try:
        async with engine.connect() as conn:
            # Query to check for checkpointer tables
            result = await conn.execute(text("""
                SELECT tablename 
                FROM pg_tables 
                WHERE schemaname = 'public' 
                AND tablename IN ('checkpoints', 'checkpoint_blobs', 'checkpoint_writes', 'checkpoint_metadata')
                ORDER BY tablename;
            """))

            tables = [row[0] for row in result]
            
            if tables:
                print("✅ Found existing checkpointer tables:")
                for table in tables:
                    print(f"   - {table}")
            else:
                print("⚠️ No checkpointer tables found. Run the setup cell above to create them.")
                
            return tables
            
    except Exception as e:
        print(f"❌ Error checking tables: {e}")
        return []
    finally:
        await engine.dispose()

# Check for existing tables
existing_tables = await check_checkpointer_tables()

📡 Checking tables in: db.cwszjkhrbfzzpjvephsx.supabase.co:5432/postgres
⚠️ No checkpointer tables found. Run the setup cell above to create them.


In [60]:
import asyncio
import os
from dotenv import load_dotenv
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver

# Load environment variables
load_dotenv()

async def setup_checkpointer():
    """
    One-time setup to initialize LangGraph checkpointer tables in PostgreSQL.
    This creates the necessary tables for storing conversation state and checkpoints.
    """
    # Build database URL from individual environment variables
    user = os.getenv("POSTGRES_USERNAME")
    password = os.getenv("POSTGRES_PASSWORD")
    host = os.getenv("POSTGRES_HOST")
    port = os.getenv("POSTGRES_PORT", "5432")
    database = os.getenv("POSTGRES_DB")
    
    # Construct the database URL (psycopg format for AsyncPostgresSaver)
    psycopg_connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    
    print(f"📡 Connecting to: {host}:{port}/{database}")
    
    try:
        # Create AsyncPostgresSaver instance using from_conn_string
        async with AsyncPostgresSaver.from_conn_string(psycopg_connection_string) as checkpointer:
            # Setup the checkpointer tables
            await checkpointer.setup()
            print("✅ LangGraph checkpointer tables initialized successfully!")
            print("   Tables created:")
            print("   - checkpoints")
            print("   - checkpoint_blobs") 
            print("   - checkpoint_writes")
            print("   - checkpoint_metadata")
            
    except Exception as e:
        print(f"❌ Failed to setup checkpointer: {e}")
        import traceback
        traceback.print_exc()

# Run the setup
await setup_checkpointer()
print("\n⚠️ This is a one-time setup. You don't need to run this cell again unless you're setting up a new database.")

📡 Connecting to: db.cwszjkhrbfzzpjvephsx.supabase.co:5432/postgres
✅ LangGraph checkpointer tables initialized successfully!
   Tables created:
   - checkpoints
   - checkpoint_blobs
   - checkpoint_writes
   - checkpoint_metadata

⚠️ This is a one-time setup. You don't need to run this cell again unless you're setting up a new database.
